# Simulating Privilege Escalation and Data Loss Using SSH and ACLs Manipulation

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

_This notebook uses alternative version of [UC7](./UC7-E2E-Demo.ipynb) which has a larger focus on PrimAITE's simulation capabilities._

## Overview

This Jupyter notebook demonstrates a cyber scenario focusing on internal privilege escalation and data loss through the manipulation of SSH access and Access Control Lists (ACLs). The scenario is designed to model and visualise how a disgruntled junior engineer might exploit internal network vulnerabilities and social engineering of account credentials to escalate privileges and cause significant data loss and disruption to services.

## Scenario Description

This simulation utilises the PrimAITE demo network, focussing specifically on five nodes:

<a href="_package_data/primaite_demo_network.png" target="_blank">
    <img src="_package_data/primaite_demo_network.png" alt="Description of Image" style="width:100%; max-width:450px;">
</a>


- **SomeTech Developer PC** (`some_tech_jnr_dev_pc`): The workstation used by the junior engineer.
- **SomeTech Core Router** (`some_tech_rt`): A critical network device that controls access between nodes.
- **SomeTech PostgreSQL Database Server** (`some_tech_db_srv`): Hosts the company’s critical database.
- **SomeTech Storage Server** (`some_tech_storage_srv`): Stores important files and database backups.
- **SomeTech Web Server** (`some_tech_web_srv`): Serves the company’s website.

By default, the junior developer PC is restricted from connecting to the storage server via FTP or SSH due to ACL rules that permit only senior members of the engineering team to access these services.

The goal of the scenario is to simulate how the junior engineer, after gaining unauthorised access to the core router, manipulates ACL rules to escalate privileges and delete critical data.

### Key Actions Simulated

1. **Privilege Escalation**: The junior engineer uses social engineering to obtain login credentials for the core router, SSHs into the router, and modifies the ACL rules to allow SSH access from their PC to the storage server.
2. **Remote Access**: The junior engineer then uses the newly gained SSH access to connect to the storage server from their PC. This step is crucial for executing further actions, such as deleting files.
3. **File Deletion**: With SSH access to the storage server, the engineer deletes the backup file from the storage server and subsequently removes critical data from the PostgreSQL database, bringing down the *sometech.ai* website.
4.  **Website Impact Verification:** After the deletion of the database backup, the scenario checks the *sometech.ai* website's status to confirm it has been brought down due to the data loss.
5.  **Database Restore Failure:** An attempt is made to restore the deleted backup, demonstrating that the restoration fails and highlighting the severity of the data loss.

### Notes:
- The demo will utilise **CAOS** (Common Action and Observation Space) actions wherever they are available. For actions where a CAOS action does not yet exist, the action will be performed manually on the node/service via the PrimAITE simulation API.
- This notebook will be updated to incorporate new CAOS actions as they become supported.

## The Scenario

In [1]:
!primaite setup

2025-03-24 09:52:18,751: Performing the PrimAITE first-time setup...
2025-03-24 09:52:18,751: Building the PrimAITE app directories...
2025-03-24 09:52:18,751: Building primaite_config.yaml...
2025-03-24 09:52:18,751: Rebuilding the demo notebooks...
2025-03-24 09:52:18,774: Rebuilding the example notebooks...
2025-03-24 09:52:18,776: PrimAITE setup complete!


In [2]:
import yaml

from primaite import PRIMAITE_PATHS
from primaite.game.game import PrimaiteGame
from primaite.simulator.file_system.folder import Folder
from primaite.utils.validation.port import PORT_LOOKUP
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.system.applications.database_client import DatabaseClient
from primaite.simulator.system.applications.web_browser import WebBrowser
from primaite.simulator.system.services.database.database_service import DatabaseService
from primaite.simulator.network.hardware.nodes.network import firewall


## Load the network configuration

In [3]:
path = PRIMAITE_PATHS.user_config_path / "example_config" / "multi_lan_internet_network_example.yaml"

with open(path, "r") as file:
    cfg = yaml.safe_load(file)

    game = PrimaiteGame.from_config(cfg)

## Capture some of the nodes from the network to observe actions

In [4]:
some_tech_jnr_dev_pc: Computer = game.simulation.network.get_node_by_hostname("some_tech_jnr_dev_pc")
some_tech_jnr_dev_db_client: DatabaseClient = some_tech_jnr_dev_pc.software_manager.software["database-client"]
some_tech_jnr_dev_web_browser: WebBrowser = some_tech_jnr_dev_pc.software_manager.software["web-browser"]
some_tech_rt: Router = game.simulation.network.get_node_by_hostname("some_tech_rt")
some_tech_db_srv: Server = game.simulation.network.get_node_by_hostname("some_tech_db_srv")
some_tech_db_service: DatabaseService = some_tech_db_srv.software_manager.software["database-service"]
some_tech_storage_srv: Server = game.simulation.network.get_node_by_hostname("some_tech_storage_srv")
some_tech_web_srv: Server = game.simulation.network.get_node_by_hostname("some_tech_web_srv")

## Perform a Database Backup and Inspect the Storage Server

At this stage, a backup of the PostgreSQL database is created and the storage server’s file system is inspected. This step ensures that a backup file is present and correctly stored in the storage server before any further actions are taken. The inspection of the file system allows verification of the backup’s existence and health, establishing a baseline that will later be used to confirm the success of the subsequent deletion actions.

In [5]:
some_tech_storage_srv.file_system.show(full=True)

+--------------------------------------------------------------------+
|                 some_tech_storage_srv File System                  |
+-----------+------+---------------+-----------------------+---------+
| File Path | Size | Health status | Visible health status | Deleted |
+-----------+------+---------------+-----------------------+---------+
| root      | 0 B  | GOOD          | NONE                  | False   |
+-----------+------+---------------+-----------------------+---------+


In [6]:
some_tech_db_service.backup_database()

True

In [7]:
some_tech_storage_srv.file_system.show(full=True)

+--------------------------------------------------------------------------------------------------------------+
|                                      some_tech_storage_srv File System                                       |
+--------------------------------------------------+---------+---------------+-----------------------+---------+
| File Path                                        | Size    | Health status | Visible health status | Deleted |
+--------------------------------------------------+---------+---------------+-----------------------+---------+
| 4c83f661-ecca-4dfe-962d-54eea590b87d/database.db | 4.77 MB | GOOD          | NONE                  | False   |
| root                                             | 0 B     | GOOD          | NONE                  | False   |
+--------------------------------------------------+---------+---------------+-----------------------+---------+


## Extract the folder name containing the database backup file

In [8]:
db_backup_folder_list = [folder.name for folder in some_tech_storage_srv.file_system.folders.values() if folder.name != "root"]
db_backup_folder = db_backup_folder_list[0]

## Check That the Junior Engineer Cannot SSH into the Storage Server

This step verifies that the junior engineer is currently restricted from SSH access to the storage server. By attempting to establish an SSH connection from the junior engineer’s workstation to the storage server, this action confirms that the current ACL rules on the core router correctly prevents unauthorised access. It sets up the necessary conditions to later validate the effectiveness of the privilege escalation by demonstrating the initial access restrictions.


In [9]:
simulation_request = [
    "network", "node", "some_tech_jnr_dev_pc", 
    "service", "terminal", "node_session_remote_login", "admin", "admin", str(some_tech_storage_srv.network_interface[1].ip_address)
]
game.simulation.apply_request(simulation_request)

RequestResponse(status='failure', data={})

## Confirm That the Website Is Up by Executing the Web Browser on the Junior Engineer's Machine

In this step, we verify that the sometech.ai website is operational before any malicious activities begin. By executing the web browser application on the junior engineer’s machine, we ensure that the website is accessible and functioning correctly. This establishes a baseline for the website’s status, allowing us to later assess the impact of the subsequent actions, such as database deletion, on the website's availability.


In [10]:
simulation_request = ["network", "node", "some_tech_jnr_dev_pc", "application", "web-browser", "execute"]
game.simulation.apply_request(simulation_request)

RequestResponse(status='success', data={})

## Exploit Core Router to Add ACL for SSH Access

At this point, the junior engineer exploits a vulnerability in the core router by obtaining the login credentials through social engineering. With SSH access to the core router, the engineer modifies the ACL rules to permit SSH connections from their machine to the storage server. This action is crucial as it will enable the engineer to remotely access the storage server and execute further malicious activities.

Interestingly, if we inspect the `active_remote_sessions` on the SomeTech core routers `UserSessionManager`, we'll see an active session appear. This active session would pop up in the observation space.

In [11]:
game.get_sim_state()["network"]["nodes"]["some_tech_rt"]["services"]["user-session-manager"]["active_remote_sessions"]

[]

In [12]:
simulation_request = [
    "network", "node", "some_tech_jnr_dev_pc", 
    "service", "terminal", "node_session_remote_login", "admin", "admin", str(some_tech_rt.network_interface[4].ip_address)
]
game.simulation.apply_request(simulation_request)

RequestResponse(status='success', data={'ip_address': '10.10.2.1', 'username': 'admin'})

In [13]:
game.get_sim_state()["network"]["nodes"]["some_tech_rt"]["services"]["user-session-manager"]["active_remote_sessions"]

['2983615f-a113-4fee-8b52-3595d15d054b']

## Inspect the ACL Table Before Adding the New Rule

Before making any changes, we first examine the current Access Control List (ACL) table on the core router. This inspection provides a snapshot of the existing rules that govern network traffic, including permissions and restrictions related to SSH access. Understanding this baseline is crucial for verifying the effect of new rules, ensuring that changes can be accurately assessed for their impact on network security and access controls.


In [14]:
some_tech_rt.acl.show()

+---------------------------------------------------------------------------------------------------------------------+
|                                           some_tech_rt Access Control List                                          |
+-------+--------+----------+-------------+--------------+----------+-------------+--------------+----------+---------+
| Index | Action | Protocol | Src IP      | Src Wildcard | Src Port | Dst IP      | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+-------------+--------------+----------+-------------+--------------+----------+---------+
| 11    | PERMIT | ANY      | 94.10.180.6 | 0.0.0.0      | 5432     | 10.10.1.11  | 0.0.0.0      | 5432     | 2       |
| 12    | PERMIT | ANY      | 10.10.1.11  | 0.0.0.0      | 5432     | 94.10.180.6 | 0.0.0.0      | 5432     | 2       |
| 13    | DENY   | ANY      | 10.10.2.12  | 0.0.0.0      | 21       | 10.10.1.12  | 0.0.0.0      | 21       | 0       |
| 14    | DENY   | ANY      | 10.10.2.12

In [15]:
simulation_request = [
    "network", "node", "some_tech_jnr_dev_pc", 
    "service", "terminal", "send_remote_command", str(some_tech_rt.network_interface[4].ip_address),
    {
        "command": [
            "acl", "add_rule", "PERMIT", "TCP",
            str(some_tech_jnr_dev_pc.network_interface[1].ip_address), "0.0.0.0", PORT_LOOKUP["SSH"],
            str(some_tech_storage_srv.network_interface[1].ip_address), "0.0.0.0", PORT_LOOKUP["SSH"],
            1
        ]
    }
]

game.simulation.apply_request(simulation_request)

RequestResponse(status='success', data={})

## Verify That the New ACL Rule Has Been Added

After updating the ACL rules on the core router, we need to confirm that the new rule has been successfully applied. This verification involves inspecting the ACL table again to ensure that the new rule allowing SSH access from the junior engineer’s PC to the storage server is present. This step is critical to ensure that the modification was executed correctly and that the junior engineer now has the intended access.

In [16]:
some_tech_rt.acl.show()

+---------------------------------------------------------------------------------------------------------------------+
|                                           some_tech_rt Access Control List                                          |
+-------+--------+----------+-------------+--------------+----------+-------------+--------------+----------+---------+
| Index | Action | Protocol | Src IP      | Src Wildcard | Src Port | Dst IP      | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+-------------+--------------+----------+-------------+--------------+----------+---------+
| 1     | PERMIT | tcp      | 10.10.2.12  | 0.0.0.0      | 22       | 10.10.1.12  | 0.0.0.0      | 22       | 0       |
| 11    | PERMIT | ANY      | 94.10.180.6 | 0.0.0.0      | 5432     | 10.10.1.11  | 0.0.0.0      | 5432     | 2       |
| 12    | PERMIT | ANY      | 10.10.1.11  | 0.0.0.0      | 5432     | 94.10.180.6 | 0.0.0.0      | 5432     | 2       |
| 13    | DENY   | ANY      | 10.10.2.12

## Terminate Remote Session on Core Router

After successfully adding the ACL rule to allow SSH access to the storage server, the junior engineer terminates the remote session on the core router. The termination of the session is a strategic move to avoid leaving an active remote login open while maintaining the newly granted access privileges for future use.

In [17]:
simulation_request = [
    "network", "node", "some_tech_jnr_dev_pc", 
    "service", "terminal", "remote_logoff", str(some_tech_rt.network_interface[4].ip_address)
]
game.simulation.apply_request(simulation_request)

RequestResponse(status='success', data={})

## Confirm the termination of the remote session

In [18]:
game.get_sim_state()["network"]["nodes"]["some_tech_rt"]["services"]["user-session-manager"]["active_remote_sessions"]

[]

## SSH into Storage Server and Delete Database Backup

With the newly added ACL rule, the junior engineer can now SSH into the storage server from their machine. The engineer proceeds to delete the critical database backup file stored on the server. This action is pivotal in the attack, as it directly impacts the availability of essential data and sets the stage for subsequent data loss and disruption of services.


In [19]:
simulation_request = [
    "network", "node", "some_tech_jnr_dev_pc", 
    "service", "terminal", "node_session_remote_login", "admin", "admin", str(some_tech_storage_srv.network_interface[1].ip_address)
]
game.simulation.apply_request(simulation_request)

RequestResponse(status='success', data={'ip_address': '10.10.1.12', 'username': 'admin'})

In [20]:
simulation_request = [
    "network", "node", "some_tech_jnr_dev_pc", 
    "service", "terminal", "send_remote_command", str(some_tech_storage_srv.network_interface[1].ip_address),
    {
        "command": [
            "file_system", "delete", "file", db_backup_folder, "database.db"
        ]
    }
]

game.simulation.apply_request(simulation_request)

RequestResponse(status='success', data={})

## Verify that the database backup file has been deleted

In [21]:
some_tech_storage_srv.file_system.show(full=True)

+--------------------------------------------------------------------------------------------------------------+
|                                      some_tech_storage_srv File System                                       |
+--------------------------------------------------+---------+---------------+-----------------------+---------+
| File Path                                        | Size    | Health status | Visible health status | Deleted |
+--------------------------------------------------+---------+---------------+-----------------------+---------+
| 4c83f661-ecca-4dfe-962d-54eea590b87d/database.db | 4.77 MB | GOOD          | NONE                  | True    |
| root                                             | 0 B     | GOOD          | NONE                  | False   |
+--------------------------------------------------+---------+---------------+-----------------------+---------+


In [22]:
# Should return 'None' as the database.db file has been deleted (as shown in the above table)
print(some_tech_storage_srv.file_system.get_file(folder_name=db_backup_folder, file_name="database.db", include_deleted=False))

None


## Delete Critical Data from the PostgreSQL Database

In this part of the scenario, the junior engineer manually interacts with the PostgreSQL database to delete critical data. The deletion of critical data from the database has significant implications, leading to the loss of essential information and affecting the availability of the sometech.ai website.

* Since the CAOS framework does not support ad-hoc or dynamic SQL queries for database services, this action must be performed manually.

##### Again, confirm that the sometech.ai website is up by executing the web browser on the junior engineer's machine

In [23]:
simulation_request = ["network", "node", "some_tech_jnr_dev_pc", "application", "web-browser", "execute"]
game.simulation.apply_request(simulation_request)

RequestResponse(status='success', data={})

##### Set the server IP address and open a new DB connection

In [24]:
some_tech_jnr_dev_db_client.server_ip_address = some_tech_db_srv.network_interface[1].ip_address
some_tech_jnr_dev_db_connection = some_tech_jnr_dev_db_client.get_new_connection()
some_tech_jnr_dev_db_connection

DatabaseClientConnection(connection_id='3a13933f-4ada-4253-bd55-514020e37b02', is_active=True)

##### Send the DELETE query

In [25]:
some_tech_jnr_dev_db_connection.query("DELETE")

True

##### Confirm that the actions have brought the sometech.ai website down

In [26]:
simulation_request = ["network", "node", "some_tech_jnr_dev_pc", "application", "web-browser", "execute"]
game.simulation.apply_request(simulation_request)

RequestResponse(status='failure', data={})

## Attempt to Restore Database Backup

In this final section, an attempt is made to restore the database backup that was deleted earlier. The action is performed using the `some_tech_db_service.restore_backup()` method. This will demonstrate the impact of the data loss and confirm that the backup restoration fails, highlighting the severity of the disruption caused.

In [27]:
some_tech_db_service.restore_backup()

False

## End of Scenario Summary

In this simulation, we modelled a cyber attack scenario where a disgruntled junior engineer exploits internal network vulnerabilities to escalate privileges, causing significant data loss and disruption of services. The following key actions were performed:

1.  **Privilege Escalation:** The junior engineer used social engineering to obtain the login credentials for the core router. They remotely accessed the router via SSH and modified the ACL rules to grant SSH access from their machine to the storage server.

2.  **Remote Access:** With the modified ACLs in place, the engineer was able to SSH into the storage server from their machine. This access enabled them to interact with the storage server and perform further actions.

3.  **File & Data Deletion:** The engineer used SSH remote access to delete a critical database backup file from the storage server. Subsequently, they executed a SQL command to delete critical data from the PostgreSQL database, which resulted in the disruption of the sometech.ai website.

4.  **Website Status Verification:** After the deletion of the database backup, the website's status was checked to confirm that it had been brought down due to the data loss.

5.  **Database Restore Failure:** An attempt to restore the deleted backup was made to demonstrate that the restoration process failed, highlighting the severity of the data loss.

**Verification and Outcomes:**

-   **Initial State Verification:** The backup file was confirmed to be present on the storage server before any actions were taken. The junior engineer's inability to SSH into the storage server initially confirmed that ACL restrictions were in effect.

-   **Privilege Escalation Confirmation:** The successful modification of the ACL rules was verified by checking the router's ACL table.

-   **Remote Access Verification:** After the ACL modification, the engineer successfully SSH'd into the storage server from their PC. The file system inspection confirmed that the backup file was accessible and could be deleted.

-   **File Deletion Confirmation:** The deletion of the backup file was confirmed by inspecting the storage server's file system after the operation. The backup file was marked as deleted, validating that the deletion command was executed.

-   **Database and Website Impact:** The deletion of the database backup was followed by a DELETE query executed on the PostgreSQL database. The website's functionality was subsequently checked using a web browser, confirming that the sometech.ai website was down due to the data loss.

-   **Restore Attempt Verification:** An attempt to restore the deleted database backup was made, and it was confirmed that the restoration failed, highlighting the impact of the data deletion.